# Pose Prediction model

In [ ]:
# Mounting Google drive to use resources from it
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
# Imports

import numpy as np
import pandas as pd
import tensorflow as tf
import ast
import re
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, BatchNormalization, MaxPooling1D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve, roc_curve
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
%matplotlib inline
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import pickle

# Loading the CSV data

In [ ]:
def dataset_compiler(csvPath):
  csv = pd.read_csv(csvPath, header=None)
  dataset = []
  labels = []
  # iterating through all image records ie rows
  for index, row in csv.iterrows():
    keypoints = []
    # iterating through each keypoint for an image
    for column in csv.columns:
        record = row[column]
        # if the record being read is the total score of the pose then just skip it
        if record[:6] == " total":
          break
        # if the record being read is the image code then add to labels array
        label = record[:3]
        if label == "HIG" or label == "KNE" or label == "LOW":
          match label:
            case "HIG":
              labels.append("High Kick")
            case "KNE":
              labels.append("Knee Strike")
            case "LOW":
              labels.append("Low Kick")
            case _:
              print("how did you get here?")
        # transforming the coordinates from the record to a numpy array
        else:
          # extracts the data to be 'x': 64; 'y': 111; 'score': 0.504564950284737
          pattern = r"{(.*?)}"
          matches = re.findall(pattern, record)
          extracted_content = matches[0] if matches else None
          # splits data by ; to have each indicidual item
          split_values = extracted_content.split(';')
          # extracts the actual number
          values = [float(val.split(':')[1]) for val in split_values]
          # adds it to the array for saving
          keypoints.append(values)
    dataset.append(keypoints)
  print("dataset compiled!")
  return dataset, labels

In [ ]:
# read from CSV
data_dir = '/content/drive/MyDrive/Colab Notebooks'
val_dir = data_dir + '/keypoints_valid.csv'
test_dir = data_dir + '/keypoints_test.csv'
train_dir = data_dir + '/keypoints_train.csv'


valid_dataset, valid_labels = dataset_compiler(val_dir)
test_dataset, test_labels = dataset_compiler(test_dir)
train_dataset, train_labels = dataset_compiler(train_dir)

dataset compiled!
dataset compiled!
dataset compiled!


# Transforming Data Arrays

In [ ]:
labels = ['High Kick', 'Knee Strike', 'Low Kick']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit label encoder and transform labels to numerical format
valid_num_labels = label_encoder.fit_transform(valid_labels)
test_num_labels = label_encoder.fit_transform(test_labels)
train_num_labels = label_encoder.fit_transform(train_labels)

In [ ]:
train_dataset_reshaped = np.reshape(train_dataset, (len(train_dataset), -1))
validation_dataset_reshaped = np.reshape(valid_dataset, (len(valid_dataset), -1))
test_dataset_reshaped = np.reshape(test_dataset, (len(test_dataset), -1))

# Print the mapping between original string labels and numerical labels (optional)

In [ ]:
def print_mapping():
    print("Train Label Mapping:")
    for train_labels, train_num_labels in zip(train_labels, train_num_labels):
        print(f"{train_labels} -> {train_num_labels}")

    print("Test Label Mapping:")
    for test_labels, test_num_labels in zip(test_labels, test_num_labels):
        print(f"{test_labels} -> {test_num_labels}")

    print("Valid Label Mapping:")
    for valid_labels, valid_num_labels in zip(valid_labels, valid_num_labels):
        print(f"{valid_labels} -> {valid_num_labels}")

print_mapping()

# Building the CNN model

In [ ]:
def create_model():
    model = tf.keras.Sequential([
        layers.Reshape((17, 3), input_shape=(17*3,)),  # Reshape input to (17 keypoints, 3 coordinates)

        # Convolutional Layers
        layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2, strides=2),

        layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2, strides=2),

        layers.Conv1D(256, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2, strides=2),

        # Flatten and Dense Layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),  # Increasing dropout rate

        layers.Dense(64, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),  # Increasing dropout rate

        layers.Dense(32, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.4),  # Increasing dropout rate

        layers.Dense(3, activation='softmax')
    ])
    return model

In [ ]:
model = create_model()

# Compile the model
opt = tf.keras.optimizers.Adam(lr=0.01)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using your training data and validate using validation data
history = model.fit(train_dataset_reshaped, np.array(train_num_labels), epochs=50, validation_data=(validation_dataset_reshaped, np.array(valid_num_labels)), callbacks=[
        keras.callbacks.EarlyStopping(patience=9, verbose=2, restore_best_weights=True),
        keras.callbacks.ReduceLROnPlateau(factor=.5, patience=3, verbose=2)])

with open("history.pkl", "wb") as file:
    pickle.dump(history.history, file)

In [ ]:
model.summary()

Model: "sequential_20"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_20 (Reshape)        (None, 17, 3)             0         
                                                                 
 conv1d_58 (Conv1D)          (None, 17, 32)            320       
                                                                 
 batch_normalization_116 (B  (None, 17, 32)            128       
 atchNormalization)                                              
                                                                 
 max_pooling1d_63 (MaxPooli  (None, 8, 32)             0         
 ng1D)                                                           
                                                                 
 conv1d_59 (Conv1D)          (None, 8, 64)             6208      
                                                                 
 batch_normalization_117 (B  (None, 8, 64)           

# Metrics for analysis

In [ ]:
train_loss = history.history["loss"]
train_accuracy = history.history["accuracy"]

val_loss = history.history["val_loss"]
val_accuracy = history.history["val_accuracy"]
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset_reshaped, np.array(test_num_labels))

print("Train Loss:", np.mean(train_loss))
print("Train Accuracy:", np.mean(train_accuracy))
print()
print("Val Loss:", np.mean(val_loss))
print("Val Accuracy:", np.mean(val_accuracy))
print()
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Make predictions and output results

In [ ]:
predictions = model.predict(x=test_dataset_reshaped, steps=len(test_dataset_reshaped), verbose=0)

In [ ]:
predicted_labels = np.argmax(predictions, axis=1)

print(classification_report(test_num_labels, predicted_labels))
print(confusion_matrix(test_num_labels, predicted_labels))

# Save The model

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks")
model.save("model_5_V3.h5") # legacy warning
model.save("model_5_V3.keras")